# Setup

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Constants
path = './data'

# Load data

In [3]:
raw_data = pd.read_pickle('./data/raw_data.pickle')
raw_data

[[{'url': 'https://api.github.com/repos/rails/rails/issues/44770',
   'repository_url': 'https://api.github.com/repos/rails/rails',
   'labels_url': 'https://api.github.com/repos/rails/rails/issues/44770/labels{/name}',
   'comments_url': 'https://api.github.com/repos/rails/rails/issues/44770/comments',
   'events_url': 'https://api.github.com/repos/rails/rails/issues/44770/events',
   'html_url': 'https://github.com/rails/rails/pull/44770',
   'id': 1181332714,
   'node_id': 'PR_kwDNIULONQu2gA',
   'number': 44770,
   'title': 'Remove override of ActiveModel#attribute_names',
   'user': {'login': 'kevindew',
    'id': 282717,
    'node_id': 'MDQ6VXNlcjI4MjcxNw==',
    'avatar_url': 'https://avatars.githubusercontent.com/u/282717?v=4',
    'gravatar_id': '',
    'url': 'https://api.github.com/users/kevindew',
    'html_url': 'https://github.com/kevindew',
    'followers_url': 'https://api.github.com/users/kevindew/followers',
    'following_url': 'https://api.github.com/users/kevindew/

In [4]:
len(raw_data)

6